<a href="https://colab.research.google.com/github/Praneet9/Visualizing-CNN-Learning-using-GRAD-CAM/blob/master/GradCAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/

In [0]:
!kaggle competitions download -c dogs-vs-cats

In [0]:
!unzip train.zip

In [0]:
!mkdir train/dog
!mkdir train/cat

In [0]:
!mv train/dog.*jpg train/dog/
!mv train/cat.*jpg train/cat/

In [0]:
!rm -r train/.ipynb_checkpoints

rm: cannot remove 'train/.ipynb_checkpoints': No such file or directory


In [0]:
from keras.layers import Conv2D, Dense, Dropout, MaxPool2D, Flatten, Input, BatchNormalization
from keras.models import Sequential, Model
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import Callback
import keras.backend as K
import cv2
import matplotlib.pyplot as plt
import os

%matplotlib inline

Using TensorFlow backend.


In [0]:
image_width = 128
image_height = 128
channels = 3
classes = 2

In [0]:
generator = ImageDataGenerator(rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    validation_split=0.10)

In [0]:
train_gen = generator.flow_from_directory('train/', class_mode='categorical', target_size=(image_height, image_width), color_mode='rgb', subset='training', batch_size=256)

test_gen = generator.flow_from_directory('train/', class_mode='categorical', target_size=(image_height, image_width), color_mode='rgb', subset='validation', batch_size=256)

Found 22500 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.


In [0]:
train_gen.class_indices

{'cat': 0, 'dog': 1}

In [0]:
def preprocess_image(image_path, width, height, channels):
  image = cv2.imread(image_path)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  image = cv2.resize(image, (width, height))
  image = image.reshape(1, width, height, channels)
  image = image / 255.
  return image

In [0]:
cat = preprocess_image('cat.jpg', image_width, image_height, channels)
dog = preprocess_image('dog.jpg', image_width, image_height, channels)
cat_dog = preprocess_image('cat_dog.jpg', image_width, image_height, channels)

In [0]:
cat.shape

(1, 128, 128, 3)

In [0]:
class Vis(Callback):

    def __init__(self, dog_idx, cat_idx, layer_idx, dog, cat, cat_dog, height, width):
      self.dog_idx = dog_idx
      self.cat_idx = cat_idx
      self.layer_idx = layer_idx
      self.width = width
      self.height = height
      self.dog = dog
      self.cat = cat
      self.cat_dog = cat_dog

    def save_heatmap(self, image, op_idx, path):
      grads = K.gradients(self.model.output[:,op_idx], self.model.layers[self.layer_idx].output)[0]

      pooled_grads = K.mean(grads,axis=(0,1,2))
      iterate = K.function([self.model.input],[pooled_grads,self.model.layers[self.layer_idx].output[0]])
      pooled_grads_value, conv_layer_output = iterate([image])

      for i in range(self.model.layers[self.layer_idx].output.shape[-1]):
        conv_layer_output[:,:,i] *= pooled_grads_value[i]
      heatmap = np.mean(conv_layer_output,axis=-1)

      for x in range(heatmap.shape[0]):
        for y in range(heatmap.shape[1]):
          heatmap[x,y] = np.max(heatmap[x,y],0)

      heatmap = np.maximum(heatmap,0)

      if np.max(heatmap) > 0.0:
        heatmap /= np.max(heatmap)

        heatmap = cv2.resize(heatmap, (self.width, self.height))
        merged = cv2.merge((heatmap, heatmap, heatmap))

        image1 = image[0]*255.
        image2 = merged*255.

        image1 = image1.astype(np.float32)
        image2 = image2.astype(np.float32)

        merged = cv2.addWeighted(image1, 0.3, image2, 0.5, 1)
        merged = cv2.cvtColor(merged, cv2.COLOR_RGB2BGR)

        path = path.split('/')

        heatmap_path = os.path.join(path[0], path[1], 'heatmap', path[2])
        cv2.imwrite(heatmap_path, image2)

        cv2.imwrite(os.path.join(path[0], path[1], path[2]), merged)

    def on_epoch_end(self, epoch, logs={}):

      self.save_heatmap(self.cat, self.cat_idx, 'visualisations/cat/{}.jpg'.format(str(epoch+1)))

      self.save_heatmap(self.dog, self.dog_idx, 'visualisations/dog/{}.jpg'.format(str(epoch+1)))

      self.save_heatmap(self.cat_dog, self.cat_idx, 'visualisations/cat_with_dog/{}.jpg'.format(str(epoch+1)))

      self.save_heatmap(self.cat_dog, self.dog_idx, 'visualisations/dog_with_cat/{}.jpg'.format(str(epoch+1)))

In [0]:
vis_callback = Vis(train_gen.class_indices['dog'], train_gen.class_indices['cat'], -8, dog, cat, cat_dog, image_height, image_width)

In [0]:
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(image_height, image_width, channels), padding='same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(128, (3, 3), strides=(1, 1), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(classes, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 128, 64)      1792      
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 128, 64)      256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 62, 62, 128)       73856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 62, 62, 128)       512       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 31, 31, 128)      

In [0]:
history = model.fit_generator(train_gen, epochs=50, callbacks=[vis_callback], validation_data=test_gen)

88/88 [==============================] - 159s 2s/step - loss: 0.9204 - acc: 0.5724 - val_loss: 0.9459 - val_acc: 0.5344
Epoch 2/50
88/88 [==============================] - 146s 2s/step - loss: 0.6597 - acc: 0.6362 - val_loss: 0.6303 - val_acc: 0.6532
Epoch 3/50
88/88 [==============================] - 147s 2s/step - loss: 0.6011 - acc: 0.6789 - val_loss: 0.7088 - val_acc: 0.6200
Epoch 4/50
88/88 [==============================] - 148s 2s/step - loss: 0.5555 - acc: 0.7123 - val_loss: 0.7152 - val_acc: 0.6412
Epoch 5/50
88/88 [==============================] - 144s 2s/step - loss: 0.5291 - acc: 0.7282 - val_loss: 0.6812 - val_acc: 0.6800
Epoch 6/50
88/88 [==============================] - 146s 2s/step - loss: 0.5116 - acc: 0.7456 - val_loss: 1.0811 - val_acc: 0.6308
Epoch 7/50
88/88 [==============================] - 146s 2s/step - loss: 0.4898 - acc: 0.7577 - val_loss: 0.7692 - val_acc: 0.6692
Epoch 8/50
88/88 [==============================] - 145s 2s/step - loss: 0.4692 - acc: 0.7723 

In [0]:
!zip -r visualisations.zip visualisations/

In [0]:
import pickle

In [0]:
pickle.dump(history, open('history.pickle', 'wb'))

In [0]:
history.history.keys()

dict_keys(['val_loss', 'val_acc', 'loss', 'acc'])